In [16]:
%pip install pandas
%pip install scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.7.1-cp310-cp310-macosx_12_0_arm64.whl (8.7 MB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Using cached scipy-1.15.3-cp310-cp310-macosx_14_0_arm64.whl (22.4 MB)
  Using cached joblib-1.5.1-py3-none-any.whl (307 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 1. 좋아요/싫어요/별점 가중치 정의
LIKE_WEIGHT = 5.0          
DISLIKE_WEIGHT = -5.0      
STAR_RATING_WEIGHTS = {  
    0.5: -5.0, 
    1.0: -3.5, 
    1.5: -2.0, 
    2.0: -1.0, 
    2.5: 0.0, 
    3.0: 0.5, 
    3.5: 1.0, 
    4.0: 2.0, 
    4.5: 4.0, 
    5.0: 5.0
}

# 2. 페르소나 분류 임계값
THRESHOLD_FOR_BABY_PERSONA = 0.1 

# 페르소나가 특정 페르소나(아기도 포함)로 분류되기 위한 최소 긍정 피드백 개수
MIN_POSITIVE_RATINGS_FOR_GROWTH = 1 
# 페르소나가 특정 페르소나(아기도 포함)로 분류되기 위한 최소 메인 페르소나 점수 
MIN_SCORE_FOR_GROWTH = 0.01 

# 페르소나 계산 시 긍정 피드백으로 간주할 최소 통합 점수
POSITIVE_FEEDBACK_THRESHOLD = 0.1 

default_personas = {
    '액션_헌터': {
        'description': '강렬한 전개에 목숨 거는 유형! 한시도 눈을 뗄 수 없는 액션, 스릴러, SF에 푹 빠지는 스타일.',
        'keywords': ['액션', '스릴러', '모험', '범죄', 'SF', '전쟁'],
        'qa_mapping': {'Q1': 'A', 'Q2': 'A', 'Q3': 'B', 'Q4': 'C', 'Q5': 'B'}
    },
    '무비_셜록': {
        'description': '디테일에 강한 분석러! 복선, 반전, 떡밥 회수에 기가 막히게 반응하는 타입.',
        'keywords': ['미스터리', '범죄', '스릴러', '드라마', '다큐멘터리', '뉴스'],
        'qa_mapping': {'Q1': 'B', 'Q2': 'C', 'Q3': 'A', 'Q4': 'D', 'Q5': 'C'}
    },
    '시네파_울보': {
        'description': '눈물과 감동에 진심인 감정이입 장인! 인물의 감정선에 깊이 공감하는 섬세한 시청자.',
        'keywords': ['드라마', '로맨스', '역사', '음악', '연속극'],
        'qa_mapping': {'Q1': 'C', 'Q2': 'D', 'Q3': 'D', 'Q4': 'B', 'Q5': 'D'}
    },
    '온기_수집가': {
        'description': '따뜻한 위로를 찾는 잔잔한 영화 큐레이터! 자극적인 건 피하고 편안하고 따뜻한 콘텐츠를 선호.',
        'keywords': ['가족', '키즈', '애니메이션', '코미디', '드라마', '리얼리티', '토크'],
        'qa_mapping': {'Q1': 'C', 'Q2': 'D', 'Q3': 'C', 'Q4': 'A', 'Q5': 'D'}
    },
    '이세계_유랑자': {
        'description': '상상의 나래를 펼치는 현실 탈출러! 판타지, SF, 애니메이션 속 방대한 세계관에 푹 빠지는 스타일.',
        'keywords': ['판타지', 'SF', '모험', '애니메이션', 'SF & 판타지'],
        'qa_mapping': {'Q1': 'D', 'Q2': 'D', 'Q3': 'B', 'Q4': 'C', 'Q5': 'B'}
    },
    '무서워도본다맨': {
        'description': '공포도 감내하는 스릴 마니아! 공포, 스릴러의 긴장감과 반전 쾌감에 중독된 타입.',
        'keywords': ['공포', '스릴러', '미스터리', '범죄'],
        'qa_mapping': {'Q1': 'D', 'Q2': 'C', 'Q3': 'C', 'Q4': 'D', 'Q5': 'C'}
    },
    '레트로_캡틴': {
        'description': '고전의 매력에 빠진 클래식 무비 마니아! 아날로그 감성과 고풍스러운 시대극의 멋을 즐김.',
        'keywords': ['서부', '역사', '다큐멘터리', '음악', '전쟁'],
        'qa_mapping': {'Q1': 'C', 'Q2': 'B', 'Q3': 'D', 'Q4': 'B', 'Q5': 'A'}
    }
}

content_genre_keywords_mapping = {
    '액션': ['액션'],
    '모험': ['모험'],
    '애니메이션': ['애니메이션'],
    '코미디': ['코미디'],
    '범죄': ['범죄'],
    '다큐멘터리': ['다큐멘터리'],
    '드라마': ['드라마'],
    '가족': ['가족'],
    '판타지': ['판타지'],
    '역사': ['역사'],
    '공포': ['공포'],
    '음악': ['음악'],
    '미스터리': ['미스터리'],
    '로맨스': ['로맨스'],
    'SF': ['SF'],
    '스릴러': ['스릴러'],
    '전쟁': ['전쟁'],
    '서부': ['서부'],
    '뉴스': ['뉴스'],
    '리얼리티': ['리얼리티'],
    '토크': ['토크'],
    '연속극': ['연속극'], 
    '키즈': ['키즈'], 
    '전쟁 & 정치': ['전쟁 & 정치'], 
    'SF & 판타지': ['SF & 판타지'],
    '(장르 없음)': ['(장르 없음)'] 
}

In [12]:
import pandas as pd
import numpy as np
import re
import os

# 1. content_data.csv 파일 로드 및 전처리
contents_df = pd.read_csv('../data_processing/content_data.csv')
contents_df_persona = contents_df[['id', 'title', 'genres']].copy()
contents_df_persona.rename(columns={'id': 'contentId'}, inplace=True)

# 장르 한글 매핑 함수
def map_genres_to_korean(genres_str):
    if pd.isna(genres_str) or genres_str == '(no genres listed)':
        return ["(장르 없음)"]
    genres = re.split(r'[|,]', genres_str)
    genres = [g.strip() for g in genres if g.strip()]
    return list(set(genres))

contents_df_persona['genres'] = contents_df_persona['genres'].apply(map_genres_to_korean)

# 제목에서 연도 정보 제거
def clean_title(title):
    return re.sub(r'\s*\(\d{4}\)$', '', title).strip()

contents_df_persona['title_cleaned'] = contents_df_persona['title'].apply(clean_title)

print("1. contents_df_persona 일부 보기:")
print(contents_df_persona.head())
print(f"총 {len(contents_df_persona)}개의 콘텐츠 정보 로드.")
print(f"contents_df_persona의 contentId (TMDB ID) 범위: {contents_df_persona['contentId'].min()} ~ {contents_df_persona['contentId'].max()}")
print(f"contents_df_persona의 고유 contentId (TMDB ID) 개수: {contents_df_persona['contentId'].nunique()}")


# 2. links.csv 로드 및 매핑 준비
links_df = pd.read_csv('../data_processing/links.csv')
# tmdbId가 NaN인 경우 제거 (매핑 불가)
links_df.dropna(subset=['tmdbId'], inplace=True)
links_df['tmdbId'] = links_df['tmdbId'].astype(int)

print("\n2. links_df 일부 보기:")
print(links_df.head())
print(f"총 {len(links_df)}개의 매핑 정보 로드.")
print(f"links_df의 movieId 범위: {links_df['movieId'].min()} ~ {links_df['movieId'].max()}")
print(f"links_df의 tmdbId 범위: {links_df['tmdbId'].min()} ~ {links_df['tmdbId'].max()}")
print(f"links_df의 고유 movieId 개수: {links_df['movieId'].nunique()}")
print(f"links_df의 고유 tmdbId 개수: {links_df['tmdbId'].nunique()}")


# 3. ratings.csv (별점 데이터) 로드 및 contentId 매핑
reviews_df = pd.read_csv('../data_processing/ratings.csv')
reviews_df.rename(columns={'rating': 'score'}, inplace=True)

initial_reviews_count = len(reviews_df)
reviews_df = pd.merge(reviews_df, links_df[['movieId', 'tmdbId']], on='movieId', how='inner')
reviews_df.drop(columns=['movieId', 'timestamp'], inplace=True)
reviews_df.rename(columns={'tmdbId': 'contentId'}, inplace=True)
reviews_df = reviews_df[['userId', 'contentId', 'score']]

# content_data.csv에 있는 contentId (tmdbId)만 남김
valid_content_ids_from_content_data = contents_df_persona['contentId'].unique()
reviews_df = reviews_df[reviews_df['contentId'].isin(valid_content_ids_from_content_data)].copy()

filtered_reviews_after_mapping_count = len(reviews_df)
print(f"\n3. reviews_df 매핑 및 필터링 완료: {initial_reviews_count}개 -> {filtered_reviews_after_mapping_count}개 레코드 (매핑 불가 또는 content_data에 없는 contentId {initial_reviews_count - filtered_reviews_after_mapping_count}개 제거)")
print("reviews_df 일부 보기:")
print(reviews_df.head())
print(f"필터링 후 reviews_df의 고유 contentId 개수: {reviews_df['contentId'].nunique()}")
if not reviews_df.empty:
    print(f"필터링 후 reviews_df의 contentId 최소/최대: {reviews_df['contentId'].min()}/{reviews_df['contentId'].max()}")


# 4. content_reactions.csv (좋아요/싫어요 데이터) 로드 및 contentId 매핑
content_reactions_df_raw = pd.read_csv('../data_processing/content_reactions.csv')
content_reactions_df_raw.rename(columns={'user_id': 'userId', 'content_id': 'contentId'}, inplace=True)
content_reactions_df_raw = content_reactions_df_raw[['userId', 'contentId', 'reaction']] 

initial_reactions_count = len(content_reactions_df_raw)

temp_reactions_df_for_merge = content_reactions_df_raw.rename(columns={'contentId': 'movieId'})
content_reactions_df = pd.merge(temp_reactions_df_for_merge, links_df[['movieId', 'tmdbId']], on='movieId', how='inner')

content_reactions_df.drop(columns=['movieId'], inplace=True)
content_reactions_df.rename(columns={'tmdbId': 'contentId'}, inplace=True) 
content_reactions_df = content_reactions_df[['userId', 'contentId', 'reaction']] 

# content_data.csv에 있는 contentId (tmdbId)만 남김
content_reactions_df = content_reactions_df[content_reactions_df['contentId'].isin(valid_content_ids_from_content_data)].copy()

filtered_reactions_after_mapping_count = len(content_reactions_df)
print(f"\n4. content_reactions_df 매핑 및 필터링 완료: {initial_reactions_count}개 -> {filtered_reactions_after_mapping_count}개 레코드 (매핑 불가 또는 content_data에 없는 contentId {initial_reactions_count - filtered_reactions_after_mapping_count}개 제거)")
print("content_reactions_df 준비 완료. 일부 보기:")
print(content_reactions_df.head())
print(f"필터링 후 content_reactions_df의 고유 contentId 개수: {content_reactions_df['contentId'].nunique()}")
if not content_reactions_df.empty:
    print(f"필터링 후 content_reactions_df의 contentId 최소/최대: {content_reactions_df['contentId'].min()}/{content_reactions_df['contentId'].max()}")

# 빈 데이터프레임 확인
if contents_df_persona.empty:
    print("[경고] 'contents_df_persona'가 비어있습니다. 이후 추천 시스템 기능이 작동하지 않을 수 있습니다.")
if reviews_df.empty:
    print("[경고] 'reviews_df'가 비어있습니다. 별점 기반 기능이 제한될 수 있습니다.")
if content_reactions_df.empty:
    print("[경고] 'content_reactions_df'가 비어있습니다. 좋아요/싫어요 기반 기능이 제한될 수 있습니다.")

1. contents_df_persona 일부 보기:
   contentId                title           genres        title_cleaned
0         11  스타워즈 에피소드 4: 새로운 희망     [모험, 액션, SF]  스타워즈 에피소드 4: 새로운 희망
1         12              니모를 찾아서      [애니메이션, 가족]              니모를 찾아서
2         13              포레스트 검프  [드라마, 코미디, 로맨스]              포레스트 검프
3         14              아메리칸 뷰티            [드라마]              아메리칸 뷰티
4         15                시민 케인      [드라마, 미스터리]                시민 케인
총 5902개의 콘텐츠 정보 로드.
contents_df_persona의 contentId (TMDB ID) 범위: 11 ~ 1513258
contents_df_persona의 고유 contentId (TMDB ID) 개수: 5851

2. links_df 일부 보기:
   movieId  imdbId  tmdbId
0        1  114709     862
1        2  113497    8844
2        3  113228   15602
3        4  114885   31357
4        5  113041   11862
총 9734개의 매핑 정보 로드.
links_df의 movieId 범위: 1 ~ 193609
links_df의 tmdbId 범위: 2 ~ 525662
links_df의 고유 movieId 개수: 9734
links_df의 고유 tmdbId 개수: 9733

3. reviews_df 매핑 및 필터링 완료: 100837개 -> 35557개 레코드 (매핑 불가 또는 content_data에 없는 conte

In [13]:
# 1. 사용자 통합 피드백 점수 생성 함수
def generate_unified_user_feedback(user_id, reactions_df, reviews_df, contents_df_persona):
    user_feedback = {}

    user_reactions = reactions_df[reactions_df['userId'] == user_id]
    user_reviews = reviews_df[reviews_df['userId'] == user_id]

    all_content_ids = pd.concat([user_reactions['contentId'], user_reviews['contentId']]).unique()
    
    content_id_to_title_map = contents_df_persona.set_index('contentId')['title_cleaned'].to_dict()

    for content_id in all_content_ids:
        content_title = content_id_to_title_map.get(content_id)
        if content_title is None:
            continue

        effective_score = 0.0
        
        # 1. 싫어요 (Dislike) 처리: 싫어요가 있으면 무조건 -5.0
        dislike_entry = user_reactions[(user_reactions['contentId'] == content_id) & (user_reactions['reaction'] == '싫어요')]
        if not dislike_entry.empty:
            effective_score = DISLIKE_WEIGHT 
        else:
            # 2. 좋아요 (Like) 처리
            like_entry = user_reactions[(user_reactions['contentId'] == content_id) & (user_reactions['reaction'] == '좋아요')]
            if not like_entry.empty:
                effective_score += LIKE_WEIGHT 

            # 3. 별점 (Star Rating) 처리
            review_entry = user_reviews[user_reviews['contentId'] == content_id]
            if not review_entry.empty:
                star_score = review_entry['score'].iloc[0]
                effective_score += STAR_RATING_WEIGHTS.get(star_score, 0.0)
        
        if effective_score != 0.0 or (not review_entry.empty and review_entry['score'].iloc[0] == 2.5):
            user_feedback[content_title] = effective_score
            
    return user_feedback

# 2. QA 기반 페르소나 점수 계산 함수
def calculate_persona_scores_from_qa(user_answers, personas_def):
    qa_scores = {persona_id: 0.0 for persona_id in personas_def.keys()}

    # Q1
    if user_answers.get('Q1') == 'A':
        qa_scores['액션_헌터'] += 1.0
    elif user_answers.get('Q1') == 'B':
        qa_scores['무비_셜록'] += 1.0
    elif user_answers.get('Q1') == 'C':
        qa_scores['시네파_울보'] += 0.6
        qa_scores['온기_수집가'] += 0.4
    elif user_answers.get('Q1') == 'D':
        qa_scores['이세계_유랑자'] += 0.7
        qa_scores['무서워도본다맨'] += 0.3
    
    # Q2
    if user_answers.get('Q2') == 'A':
        qa_scores['액션_헌터'] += 1.0
    elif user_answers.get('Q2') == 'B':
        qa_scores['레트로_캡틴'] += 0.7
        qa_scores['온기_수집가'] += 0.3
    elif user_answers.get('Q2') == 'C':
        qa_scores['무비_셜록'] += 0.7
        qa_scores['무서워도본다맨'] += 0.3
    elif user_answers.get('Q2') == 'D':
        qa_scores['시네파_울보'] += 0.6
        qa_scores['온기_수집가'] += 0.4

    # Q3
    if user_answers.get('Q3') == 'A':
        qa_scores['무비_셜록'] += 1.0
    elif user_answers.get('Q3') == 'B':
        qa_scores['액션_헌터'] += 0.6
        qa_scores['이세계_유랑자'] += 0.4
    elif user_answers.get('Q3') == 'C':
        qa_scores['무서워도본다맨'] += 0.7
        qa_scores['온기_수집가'] += 0.3
    elif user_answers.get('Q3') == 'D':
        qa_scores['온기_수집가'] += 0.6
        qa_scores['시네파_울보'] += 0.4

    # Q4
    if user_answers.get('Q4') == 'A':
        qa_scores['온기_수집가'] += 1.0
    elif user_answers.get('Q4') == 'B':
        qa_scores['시네파_울보'] += 0.6
        qa_scores['레트로_캡틴'] += 0.4
    elif user_answers.get('Q4') == 'C':
        qa_scores['이세계_유랑자'] += 0.7
        qa_scores['액션_헌터'] += 0.3
    elif user_answers.get('Q4') == 'D':
        qa_scores['무서워도본다맨'] += 0.6
        qa_scores['무비_셜록'] += 0.4

    # Q5
    if user_answers.get('Q5') == 'A':
        qa_scores['레트로_캡틴'] += 0.7
        qa_scores['시네파_울보'] += 0.3
    elif user_answers.get('Q5') == 'B':
        qa_scores['이세계_유랑자'] += 0.6
        qa_scores['액션_헌터'] += 0.4
    elif user_answers.get('Q5') == 'C':
        qa_scores['무비_셜록'] += 0.7
        qa_scores['무서워도본다맨'] += 0.3
    elif user_answers.get('Q5') == 'D':
        qa_scores['시네파_울보'] += 0.7
        qa_scores['온기_수집가'] += 0.3

    return qa_scores

# 3. 콘텐츠 평가 이력 기반 페르소나 점수 계산 함수
def calculate_persona_scores_from_contents_genres(unified_user_feedback, contents_df_persona, personas_def, content_genre_keywords_map, positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD):
    persona_scores = {persona_id: 0.0 for persona_id in personas_def.keys()}
    
    title_to_genres_map = contents_df_persona.set_index('title_cleaned')['genres'].to_dict()

    for content_title, feedback_score in unified_user_feedback.items():
        # 긍정적인 피드백만 페르소나 점수 계산에 사용
        if feedback_score > positive_feedback_threshold:
            content_genres = title_to_genres_map.get(content_title)
            if content_genres: # 장르 정보가 있는 경우
                for persona_id, persona_info in personas_def.items():
                    persona_keywords = persona_info['keywords']
                    
                    # 콘텐츠의 각 장르가 페르소나의 키워드에 해당하는지 확인
                    for genre in content_genres:
                        for keyword in persona_keywords:
                            mapped_genres = content_genre_keywords_map.get(keyword, [])
                            if genre in mapped_genres:
                                persona_scores[persona_id] += feedback_score

    # 점수 정규화
    total_score = sum(persona_scores.values())
    if total_score > 0:
        for persona_id in persona_scores:
            persona_scores[persona_id] /= total_score
    
    return persona_scores

# 4. 하이브리드 페르소나 결정 (QA + 콘텐츠 피드백)
def get_hybrid_persona(user_answers, unified_user_feedback, contents_df_persona, personas_def, content_genre_keywords_map, qa_weight_factor=0.5, min_positive_ratings_for_growth=MIN_POSITIVE_RATINGS_FOR_GROWTH, min_score_for_growth=MIN_SCORE_FOR_GROWTH, positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD):
    # 1. QA 기반 페르소나 점수 계산
    qa_persona_scores = calculate_persona_scores_from_qa(user_answers, personas_def)
    
    # 2. 콘텐츠 기반 페르소나 점수 계산
    content_persona_scores = calculate_persona_scores_from_contents_genres(unified_user_feedback, contents_df_persona, personas_def, content_genre_keywords_map, positive_feedback_threshold)
    
    hybrid_persona_scores = {p: 0.0 for p in personas_def.keys()}

    # 3. 하이브리드 점수 계산
    for p in personas_def.keys():
        hybrid_persona_scores[p] = (qa_persona_scores.get(p, 0.0) * qa_weight_factor) + \
                                   (content_persona_scores.get(p, 0.0) * (1 - qa_weight_factor))

    # 4. 가장 높은 점수의 페르소나 결정
    if not hybrid_persona_scores:
        return "모호한_관찰자", 0.0, {}, None, 0.0, "모호한"

    # 점수 기준으로 내림차순 정렬
    sorted_personas_by_score = sorted(hybrid_persona_scores.items(), key=lambda item: item[1], reverse=True)
    
    # 최고 점수 확인
    if not sorted_personas_by_score:
        return "모호한_관찰자", 0.0, {}, None, 0.0, "모호한"

    highest_score = sorted_personas_by_score[0][1]
    
    # 최고 점수를 받은 모든 페르소나 필터링
    top_score_personas = [item for item in sorted_personas_by_score if item[1] == highest_score]

    # 동점 페르소나가 여러 개일 경우 추가 판단 로직
    if len(top_score_personas) > 1:
        highest_rated_content_id = None
        max_rating = -1.0 
        
        for title_cleaned, score in unified_user_feedback.items():
            if score > 0:
                if score > max_rating:
                    max_rating = score
                    content_info = contents_df_persona[contents_df_persona['title_cleaned'] == title_cleaned]
                    if not content_info.empty:
                        highest_rated_content_id = content_info['contentId'].iloc[0]
                    else:
                        highest_rated_content_id = None 

        # 최고 별점 작품이 있을 경우
        if highest_rated_content_id is not None:
            best_content_genres = contents_df_persona[contents_df_persona['contentId'] == highest_rated_content_id]['genres'].iloc[0]
            
            # 각 동점 페르소나별 장르 매칭 점수 계산
            persona_match_counts = {p_name: 0 for p_name, _ in top_score_personas}
            
            for p_name, _ in top_score_personas:
                persona_keywords = set()
                if p_name in personas_def:
                    for genre_keyword in personas_def[p_name]['keywords']:
                        for content_genre, mapped_keywords in content_genre_keywords_map.items():
                            if genre_keyword in mapped_keywords:
                                persona_keywords.add(content_genre)
                
                for content_genre in best_content_genres:
                    if content_genre in persona_keywords:
                        persona_match_counts[p_name] += 1
            
            # 매칭 개수가 가장 높은 페르소나 선택
            if any(count > 0 for count in persona_match_counts.values()):
                sorted_by_matches = sorted(persona_match_counts.items(), key=lambda item: (-item[1], item[0]))
                main_persona = sorted_by_matches[0][0]
                main_score = highest_score
                
                # 서브 페르소나 가져오기
                remaining_personas = [p for p in sorted_personas_by_score if p[0] != main_persona]
                if remaining_personas:
                    sub_persona = remaining_personas[0][0]
                    sub_score = remaining_personas[0][1]
                else:
                    sub_persona = None
                    sub_score = 0.0
                
            else:
                # 최고 별점 작품이 어떤 동점 페르소나 장르와도 매칭되지 않는 경우
                # 기존의 알파벳 순 정렬 fallback (점수가 같으면 이름으로)
                final_sorted_top_personas = sorted(top_score_personas, key=lambda item: item[0])
                main_persona = final_sorted_top_personas[0][0]
                main_score = final_sorted_top_personas[0][1]
                if len(final_sorted_top_personas) > 1:
                    sub_persona = final_sorted_top_personas[1][0]
                    sub_score = final_sorted_top_personas[1][1]
                else:
                    sub_persona = None
                    sub_score = 0.0

        else: 
            # 최고 별점 작품이 없는 경우 (피드백이 없거나 모두 0 이하인 경우)
            # 기존의 알파벳 순 정렬 fallback
            final_sorted_top_personas = sorted(top_score_personas, key=lambda item: item[0])
            main_persona = final_sorted_top_personas[0][0]
            main_score = final_sorted_top_personas[0][1]
            if len(final_sorted_top_personas) > 1:
                sub_persona = final_sorted_top_personas[1][0]
                sub_score = final_sorted_top_personas[1][1]
            else:
                sub_persona = None
                sub_score = 0.0
    else: # 동점 페르소나가 없는 경우 (가장 높은 점수가 유일한 경우)
        main_persona = top_score_personas[0][0]
        main_score = top_score_personas[0][1]
        
        # 서브 페르소나는 최고 점수 페르소나를 제외하고 다음으로 높은 점수
        remaining_personas = [p for p in sorted_personas_by_score if p[0] != main_persona]
        if remaining_personas:
            sub_persona = remaining_personas[0][0]
            sub_score = remaining_personas[0][1]
        else:
            sub_persona = None
            sub_score = 0.0


    # 페르소나 상태 결정 로직
    total_positive_feedback_items = sum(1 for score in unified_user_feedback.values() if score > positive_feedback_threshold)
    
    persona_status_text = ""
    if total_positive_feedback_items >= min_positive_ratings_for_growth and main_score >= min_score_for_growth:
        if sub_persona and (main_score - sub_score < THRESHOLD_FOR_BABY_PERSONA):
            persona_status_text = "아기 "
        else:
            persona_status_text = ""
    else:
        persona_status_text = "모호한"

    return main_persona, main_score, hybrid_persona_scores, sub_persona, sub_score, persona_status_text

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

# 1. 콘텐츠 유사도 매트릭스 생성

# contents_df_persona의 장르 리스트를 원-핫 인코딩
all_genres = sorted(list(set(genre for sublist in contents_df_persona['genres'] for genre in sublist)))
genre_matrix = pd.DataFrame(0, index=contents_df_persona['contentId'], columns=all_genres)

for index, row in contents_df_persona.iterrows():
    content_id = row['contentId']
    for genre in row['genres']:
        if genre in genre_matrix.columns: 
            genre_matrix.loc[content_id, genre] = 1

# 코사인 유사도 계산
content_similarity_matrix = cosine_similarity(genre_matrix)
content_similarity_df = pd.DataFrame(content_similarity_matrix, 
                                     index=contents_df_persona['contentId'], 
                                     columns=contents_df_persona['contentId'])

print("\n콘텐츠 유사도 매트릭스 (content_similarity_df) 보기:\n", content_similarity_df.iloc[:5, :5])
print(f"매트릭스 크기: {content_similarity_df.shape[0]}x{content_similarity_df.shape[1]}")

# 2. 콘텐츠 기반 추천 함수
def recommend_contents_content_based(target_content_ids, content_similarity_df, contents_df, num_recommendations=10):
    if not isinstance(target_content_ids, list):
        target_content_ids = [target_content_ids]

    content_info_map = contents_df.set_index('contentId')[['title', 'title_cleaned', 'genres']].to_dict('index')

    # 사용자가 이미 본 콘텐츠와 동일한 콘텐츠는 추천에서 제외
    excluded_content_ids = set(target_content_ids)
    
    # 각 타겟 콘텐츠와 다른 콘텐츠들의 유사도를 합산
    summed_similarities = pd.Series(0.0, index=content_similarity_df.index)
    
    valid_target_ids = []
    for content_id in target_content_ids:
        if content_id in content_similarity_df.index:
            summed_similarities += content_similarity_df[content_id]
            valid_target_ids.append(content_id)
        else:
            print(f"[경고] 타겟 콘텐츠 ID {content_id}가 유사도 매트릭스에 없습니다. 추천에서 제외합니다.")

    if not valid_target_ids:
        print("[경고] 유효한 타겟 콘텐츠 ID가 없습니다. 콘텐츠 기반 추천을 수행할 수 없습니다.")
        return []

    # 이미 본 콘텐츠 제외
    summed_similarities = summed_similarities.drop(labels=excluded_content_ids, errors='ignore')

    # 유사도 높은 순으로 정렬
    recommended_content_ids = summed_similarities.sort_values(ascending=False).index.tolist()

    recommendations = []
    for content_id in recommended_content_ids:
        if len(recommendations) >= num_recommendations:
            break

        content_info = content_info_map.get(content_id)
        if content_info:
            recommendations.append({
                'title': content_info['title'],
                'genres': ', '.join(content_info['genres']) if isinstance(content_info['genres'], list) else content_info['genres'],
                'similarity': summed_similarities.loc[content_id] 
            })
            
    return recommendations


콘텐츠 유사도 매트릭스 (content_similarity_df) 보기:
 contentId   11   12        13        14        15
contentId                                        
11         1.0  0.0  0.000000  0.000000  0.000000
12         0.0  1.0  0.000000  0.000000  0.000000
13         0.0  0.0  1.000000  0.577350  0.408248
14         0.0  0.0  0.577350  1.000000  0.707107
15         0.0  0.0  0.408248  0.707107  1.000000
매트릭스 크기: 5902x5902


/Users/jimin/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/jimin/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/jimin/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [18]:
user_persona_csv_filename = '../data_processing/user_personas.csv' 

user_persona_df_from_db = pd.DataFrame() 

if not os.path.exists(os.path.dirname(user_persona_csv_filename)) and os.path.dirname(user_persona_csv_filename) != '':
    os.makedirs(os.path.dirname(user_persona_csv_filename))
    print(f"'{os.path.dirname(user_persona_csv_filename)}' 디렉토리가 없어서 생성했습니다.")

if os.path.exists(user_persona_csv_filename):
    print(f"[알림] '{user_persona_csv_filename}' 파일에서 사용자 페르소나 데이터를 로드합니다.")
    user_persona_df_from_db = pd.read_csv(user_persona_csv_filename)
    user_persona_df_from_db['user_id'] = user_persona_df_from_db['user_id'].astype(int)
else:
    print(f"[경고] '{user_persona_csv_filename}' 파일이 존재하지 않습니다. 데이터를 동적으로 생성합니다. (최초 1회 실행 후 저장 권장)")
    
    unique_user_ids_for_persona_generation = pd.concat([reviews_df['userId'], content_reactions_df['userId']]).unique() 
    unique_user_ids_for_persona_generation.sort() 

    all_user_personas_data_for_db = []
    
    print(f"총 {len(unique_user_ids_for_persona_generation)}명의 사용자에 대해 페르소나 동적 생성 시작...")
    for uid in unique_user_ids_for_persona_generation:
        user_unified_feedback_dict = generate_unified_user_feedback(uid, content_reactions_df, reviews_df, contents_df_persona)
        
        main_persona, main_score, all_scores_dict, sub_persona_name, sub_persona_score, persona_status_text = get_hybrid_persona(
            {}, 
            user_unified_feedback_dict, 
            contents_df_persona, 
            personas_def=default_personas, 
            content_genre_keywords_map=content_genre_keywords_mapping, 
            qa_weight_factor=0.0, 
            min_positive_ratings_for_growth=MIN_POSITIVE_RATINGS_FOR_GROWTH, 
            min_score_for_growth=MIN_SCORE_FOR_GROWTH, 
            positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD 
        )
        
        for persona_id, score in all_scores_dict.items():
            all_user_personas_data_for_db.append({
                'user_id': uid, 
                'persona_id': persona_id,
                'score': score
            })
    
    user_persona_df_from_db = pd.DataFrame(all_user_personas_data_for_db)
    
    if not user_persona_df_from_db.empty:
        user_persona_df_from_db.to_csv(user_persona_csv_filename, index=False)
        print(f"생성된 페르소나 데이터가 '{user_persona_csv_filename}' 파일로 저장되었습니다.")
    else:
        print("[경고] 동적으로 생성된 페르소나 데이터가 비어있습니다.")

if user_persona_df_from_db.empty:
    print("[경고] 'user_persona_df_from_db'가 비어있습니다. 페르소나 기반 추천이 제한될 수 있습니다.")
else:
    print(f"'user_persona_df_from_db' 준비 완료. 총 {len(user_persona_df_from_db['user_id'].unique())}명 사용자의 페르소나 정보 포함.")
    print(user_persona_df_from_db.head())

# 1. 통합 선호도 데이터프레임 생성 (좋아요/싫어요/별점 합산)
print("\n좋아요/싫어요/별점을 통합하여 협업 필터링용 데이터 생성 중...")

contents_df_cf_for_merge = contents_df_persona[['contentId', 'title_cleaned', 'genres']].copy()

movie_id_to_title_for_cf = contents_df_persona.set_index('contentId')['title_cleaned'].to_dict()

temp_reviews_df = reviews_df[['userId', 'contentId', 'score']].copy() 
temp_reviews_df.rename(columns={'contentId': 'movieId'}, inplace=True) 
temp_reviews_df['effective_score'] = temp_reviews_df['score'].map(STAR_RATING_WEIGHTS).fillna(0.0)

temp_reactions_df = content_reactions_df[['userId', 'contentId', 'reaction']].copy()
temp_reactions_df.rename(columns={'contentId': 'movieId'}, inplace=True) 
temp_reactions_df['reaction_score'] = temp_reactions_df['reaction'].map({'좋아요': LIKE_WEIGHT, '싫어요': DISLIKE_WEIGHT}).fillna(0.0)

# 2. 사용자-콘텐츠별 통합 점수 계산 (싫어요 최우선)
unified_cf_data = []

all_user_movie_pairs = pd.concat([
    temp_reviews_df[['userId', 'movieId']],
    temp_reactions_df[['userId', 'movieId']]
]).drop_duplicates()

for idx, row in all_user_movie_pairs.iterrows():
    uid = row['userId']
    mid = row['movieId']
    
    effective_score = 0.0
    
    dislike_exists = temp_reactions_df[
        (temp_reactions_df['userId'] == uid) & 
        (temp_reactions_df['movieId'] == mid) & 
        (temp_reactions_df['reaction'] == '싫어요')
    ]
    if not dislike_exists.empty:
        effective_score = DISLIKE_WEIGHT
    else:
        like_row = temp_reactions_df[
            (temp_reactions_df['userId'] == uid) & 
            (temp_reactions_df['movieId'] == mid) & 
            (temp_reactions_df['reaction'] == '좋아요')
        ]
        if not like_row.empty:
            effective_score += like_row['reaction_score'].iloc[0]

        review_row = temp_reviews_df[
            (temp_reviews_df['userId'] == uid) & 
            (temp_reviews_df['movieId'] == mid)
        ]
        if not review_row.empty:
            effective_score += review_row['effective_score'].iloc[0]
    
    if effective_score != 0.0 or (not review_row.empty and review_row['score'].iloc[0] == 2.5): 
         unified_cf_data.append({'userId': uid, 'movieId': mid, 'unified_score': effective_score})

cf_df = pd.DataFrame(unified_cf_data)

if cf_df.empty:
    print("[경고] 통합된 협업 필터링 데이터가 비어있습니다. 협업 필터링을 수행할 수 없습니다.")
    user_content_matrix_cf = pd.DataFrame() 
    user_similarity_df_cf = pd.DataFrame() 
    all_user_personas_df = pd.DataFrame() 
else:
    user_content_matrix_cf = cf_df.pivot_table(index='userId', columns='movieId', values='unified_score').fillna(0)
    user_content_matrix_cf.columns.name = 'movieId' 

    print("\n통합 점수 기반 사용자-콘텐츠 행렬 (user_content_matrix_cf) 생성 완료. 일부 보기:\n", user_content_matrix_cf.iloc[:5, :5])
    print(f"행렬 크기: {user_content_matrix_cf.shape[0]} 사용자, {user_content_matrix_cf.shape[1]} 콘텐츠")

    # 3. 사용자 유사도 계산 (피어슨 상관 계수 사용)
    user_similarity_df_cf = user_content_matrix_cf.T.corr(method='pearson', min_periods=1)
    user_similarity_df_cf.fillna(0, inplace=True) 
    
    print("\n사용자 유사도 행렬 (user_similarity_df_cf) 생성 완료 (피어슨 상관 계수). 일부 보기:\n", user_similarity_df_cf.iloc[:5, :5])

    # 4. 모든 사용자의 메인 페르소나 계산
    all_user_personas_data_final = []
    for uid in user_persona_df_from_db['user_id'].unique(): 
        user_persona_scores = user_persona_df_from_db[user_persona_df_from_db['user_id'] == uid].copy()
        user_persona_scores.sort_values(by='score', ascending=False, inplace=True)

        main_persona_name = "모호한_관찰자"
        main_persona_score = 0.0
        sub_persona_name = None
        sub_persona_score = 0.0
        persona_status_text = "모호한"

        if not user_persona_scores.empty:
            main_persona_name = user_persona_scores.iloc[0]['persona_id']
            main_persona_score = user_persona_scores.iloc[0]['score']

            if len(user_persona_scores) > 1:
                sub_persona_name = user_persona_scores.iloc[1]['persona_id']
                sub_persona_score = user_persona_scores.iloc[1]['score']
            
            score_difference = main_persona_score - (sub_persona_score if sub_persona_name else 0.0)
            
            if main_persona_score >= MIN_SCORE_FOR_GROWTH:
                if sub_persona_name and (score_difference < THRESHOLD_FOR_BABY_PERSONA): 
                    persona_status_text = f"아기 {main_persona_name}"
                else: 
                    persona_status_text = main_persona_name
            else: 
                persona_status_text = "모호한"

        all_user_personas_data_final.append({
            'userId': uid,
            'main_persona': main_persona_name,
            'main_persona_score': main_persona_score,
            'sub_persona': sub_persona_name,
            'sub_persona_score': sub_persona_score,
            'persona_status': persona_status_text
        })
    
    all_user_personas_df = pd.DataFrame(all_user_personas_data_final)

    if all_user_personas_df.empty:
        print("[경고] 모든 사용자의 페르소나 정보가 생성되지 않았습니다. 협업 필터링 추천이 제대로 동작하지 않을 수 있습니다.")
    else:
        print("\n모든 사용자의 메인 페르소나 및 페르소나 상태 계산 완료. 일부 보기:\n", all_user_personas_df.head())
        print("\n--- 페르소나 상태별 사용자 수 ---")
        print(all_user_personas_df['persona_status'].value_counts())
        print("---------------------------------")


    # 5. 평점 예측 함수 (유사 사용자의 가중 평균 사용 - 피어슨 기반)
    def predict_rating_cf(user_id, content_id, user_content_matrix, relevant_similarities_filtered):
        if user_id not in user_content_matrix.index or content_id not in user_content_matrix.columns:
            return np.nan 

        rated_by_others = user_content_matrix.loc[relevant_similarities_filtered.index, content_id]
        rated_by_others = rated_by_others[rated_by_others != 0]
        
        if rated_by_others.empty:
            return np.nan 

        final_relevant_similarities = relevant_similarities_filtered.loc[rated_by_others.index]
        
        if final_relevant_similarities.sum() == 0:
            return np.nan 

        predicted_rating = (rated_by_others * final_relevant_similarities).sum() / final_relevant_similarities.sum()
        return predicted_rating

    # 6. 협업 필터링 추천 함수 (페르소나 기반 유사 사용자 필터링 및 장르 우선순위)
    def recommend_contents_cf(target_user_id, user_content_matrix, user_similarity_df, contents_df_cf, all_user_personas_df, default_personas, content_genre_keywords_mapping, num_recommendations=10, excluded_content_ids=None):
        if target_user_id not in user_content_matrix.index:
            print(f"[정보] 대상 사용자 ID {target_user_id}는 아직 평가 이력이 없어 협업 필터링이 불가능합니다.")
            return [] # 사용자 데이터가 없으면 바로 빈 리스트 반환
            
        user_rated_content_ids = user_content_matrix.loc[target_user_id, user_content_matrix.loc[target_user_id, :] != 0].index.tolist()
        
        if excluded_content_ids is None:
            final_excluded_ids = set(user_rated_content_ids)
        else:
            final_excluded_ids = set(user_rated_content_ids + excluded_content_ids)
            
        if target_user_id not in user_similarity_df.index:
            print(f"[경고] 대상 사용자 ID {target_user_id}가 유사도 행렬에 없습니다.")
            return [] 

        similar_users = user_similarity_df[target_user_id].drop(target_user_id).sort_values(ascending=False)
        similar_users = similar_users[similar_users > 0.1] 

        if similar_users.empty:
            print(f"[정보] 사용자 {target_user_id}와 유사한 사용자를 찾을 수 없습니다. (모든 유사도 0.1 이하)")
            return [] 

        predicted_ratings = {}
        all_content_ids = user_content_matrix.columns

        for content_id in all_content_ids:
            if content_id in final_excluded_ids:
                continue

            users_who_rated_content = user_content_matrix.loc[similar_users.index, content_id]
            users_who_rated_content = users_who_rated_content[users_who_rated_content != 0]

            if users_who_rated_content.empty:
                continue

            numerator = (users_who_rated_content * similar_users[users_who_rated_content.index]).sum()
            denominator = similar_users[users_who_rated_content.index].sum()

            if denominator != 0:
                predicted_ratings[content_id] = numerator / denominator
        
        sorted_predictions = sorted(predicted_ratings.items(), key=lambda item: item[1], reverse=True)

        target_persona_info = all_user_personas_df[all_user_personas_df['userId'] == target_user_id]
        target_main_persona = None
        target_persona_keywords = set()

        if not target_persona_info.empty:
            target_main_persona = target_persona_info['main_persona'].iloc[0]
            if target_main_persona in default_personas:
                for genre_keyword in default_personas[target_main_persona]['keywords']:
                    for content_genre, mapped_keywords in content_genre_keywords_mapping.items():
                        if genre_keyword in mapped_keywords:
                            target_persona_keywords.add(content_genre)
        
        PENALTY_FOR_NO_PERSONA_MATCH = -1.5 

        processed_recommendations = []

        for content_id, predicted_rating_raw in sorted_predictions:
            content_info = contents_df_cf[contents_df_cf['contentId'] == content_id]
            if not content_info.empty:
                title = content_info['title_cleaned'].iloc[0]
                genres = content_info['genres'].iloc[0]

                is_persona_genre_match = False
                
                if target_main_persona and target_persona_keywords:
                    for content_genre in genres:
                        if content_genre in target_persona_keywords:
                            is_persona_genre_match = True
                            break

                    if is_persona_genre_match and target_main_persona == '시네파_울보':
                        if '액션' in genres or '범죄' in genres or '스릴러' in genres:
                            is_persona_genre_match = False

                final_predicted_rating = predicted_rating_raw
                if not is_persona_genre_match:
                    final_predicted_rating += PENALTY_FOR_NO_PERSONA_MATCH
                
                rec_item = {
                    'title': title,
                    'genres': ', '.join(genres),
                    'predicted_rating': final_predicted_rating,
                    'persona_genre_match': is_persona_genre_match
                }
                processed_recommendations.append(rec_item)

        final_recommendations_sorted = sorted(processed_recommendations, key=lambda x: x['predicted_rating'], reverse=True)
        
        return final_recommendations_sorted[:num_recommendations]

[알림] '../data_processing/user_personas.csv' 파일에서 사용자 페르소나 데이터를 로드합니다.
'user_persona_df_from_db' 준비 완료. 총 611명 사용자의 페르소나 정보 포함.
   user_id persona_id     score
0        1      액션_헌터  0.194824
1        1    이세계_유랑자  0.136668
2        1      무비_셜록  0.173888
3        1    무서워도본다맨  0.032858
4        1     시네파_울보  0.126781

좋아요/싫어요/별점을 통합하여 협업 필터링용 데이터 생성 중...

통합 점수 기반 사용자-콘텐츠 행렬 (user_content_matrix_cf) 생성 완료. 일부 보기:
 movieId   11   12   13   14   15
userId                          
1        5.0  0.0  2.0  5.0  5.0
2        0.0  0.0  0.0  0.0  0.0
3        5.0  0.0  0.0  0.0  0.0
4        5.0  0.0  0.0  5.0  0.0
5        0.0  0.0  0.0  0.0  0.0
행렬 크기: 611 사용자, 1099 콘텐츠

사용자 유사도 행렬 (user_similarity_df_cf) 생성 완료 (피어슨 상관 계수). 일부 보기:
 userId         1         2         3         4         5
userId                                                  
1       1.000000  0.029537  0.022341  0.155018  0.080059
2       0.029537  1.000000  0.004199 -0.060951 -0.007564
3       0.022341  0.004199  1.00000

In [19]:
# 시나리오 1: 새로운 사용자의 질문지 답변 및 좋아하는 작품 3개 선택 시뮬레이션

user_answers_s1 = {
    "Q1": "A", # 화끈한 액션
    "Q2": "A", # 예상치 못한 반전과 긴장감 (액션_헌터)
    "Q3": "B", # 새로운 세계에서 모험 (이세계_유랑자, 액션_헌터)
    "Q4": "C", # 낯선 세상 탐험 (이세계_유랑자, 액션_헌터)
    "Q5": "B"  # 미지의 행성에서 생존 (이세계_유랑자, 액션_헌터)
}

user_preferred_titles_s1_likes = [
    "스타워즈 에피소드 4: 새로운 희망", 
    "메트로폴리스",                  
    "니모를 찾아서"                     
]

print("새로운 사용자가 질문지에 답변하고 좋아하는 작품을 선택 중...")

if not contents_df_persona.empty:
    temp_user_id_s1 = 9999999

    virtual_reviews_df_s1 = pd.DataFrame(columns=['userId', 'contentId', 'score']) 

    virtual_content_reactions_df_s1_data = []
    excluded_content_ids_s1 = [] 

    for title in user_preferred_titles_s1_likes:
        content_info = contents_df_persona[contents_df_persona['title_cleaned'] == clean_title(title)] 
        if not content_info.empty:
            content_id = content_info['contentId'].iloc[0]
            virtual_content_reactions_df_s1_data.append({
                'userId': temp_user_id_s1, 
                'contentId': content_id, 
                'reaction': '좋아요'
            })
            excluded_content_ids_s1.append(content_id) 

    virtual_content_reactions_df_s1 = pd.DataFrame(virtual_content_reactions_df_s1_data)
    
    unified_user_feedback_s1 = generate_unified_user_feedback(
        temp_user_id_s1, 
        virtual_content_reactions_df_s1, 
        virtual_reviews_df_s1,           
        contents_df_persona              
    )
    
    print(f"\n생성된 통합 사용자 피드백 (시나리오 1): {unified_user_feedback_s1}")

    main_persona_s1, main_score_s1, all_scores_s1, sub_persona_s1, sub_score_s1, main_status_text_s1 = get_hybrid_persona(
        user_answers_s1, 
        unified_user_feedback_s1, 
        contents_df_persona, 
        default_personas, 
        content_genre_keywords_mapping, 
        qa_weight_factor=0.5, 
        min_positive_ratings_for_growth=MIN_POSITIVE_RATINGS_FOR_GROWTH, 
        min_score_for_growth=MIN_SCORE_FOR_GROWTH, 
        positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD 
    )

    print(f"\n✅ 시나리오 1 결과:")
    print(f"메인 페르소나: '{main_status_text_s1} {main_persona_s1}' (점수: {main_score_s1:.2f})")
    if sub_persona_s1:
        print(f"서브 페르소나: '{sub_persona_s1}' (점수: {sub_score_s1:.2f})")
    print("전체 페르소나 점수:", {k: f"{v:.2f}" for k, v in all_scores_s1.items()})

    # --------------------------------------------------------------------------
    # 협업 필터링을 위한 임시 사용자 정보 추가 및 추천
    # (다음 시나리오를 위해 current_ 변수에 저장)
    # --------------------------------------------------------------------------

    new_user_persona_data_s1 = {
        'userId': temp_user_id_s1,
        'main_persona': main_persona_s1,
        'main_persona_score': main_score_s1,
        'sub_persona': sub_persona_s1,
        'sub_persona_score': sub_score_s1,
        'persona_status': main_status_text_s1 
    }
    # 기존 all_user_personas_df에 새 사용자 추가
    current_all_user_personas_df = pd.concat([all_user_personas_df, pd.DataFrame([new_user_persona_data_s1])], ignore_index=True)

    # 새로운 사용자의 통합 피드백을 user_content_matrix_cf에 반영
    current_user_content_matrix_cf = user_content_matrix_cf.copy()

    if temp_user_id_s1 not in current_user_content_matrix_cf.index:
        current_user_content_matrix_cf.loc[temp_user_id_s1] = 0.0

    for title_cleaned, score in unified_user_feedback_s1.items():
        content_id_row = contents_df_persona[contents_df_persona['title_cleaned'] == title_cleaned]
        if not content_id_row.empty:
            content_id = content_id_row['contentId'].iloc[0]
            if content_id in current_user_content_matrix_cf.columns:
                current_user_content_matrix_cf.loc[temp_user_id_s1, content_id] = score
            else:
                # 새로운 콘텐츠인 경우 컬럼 추가 및 값 할당
                current_user_content_matrix_cf[content_id] = 0.0
                current_user_content_matrix_cf.loc[temp_user_id_s1, content_id] = score
                
    print("\n새로운 사용자 포함하여 사용자 유사도 행렬 재계산 중 (시나리오 1)...")
    current_user_similarity_df_cf = pd.DataFrame(
        np.corrcoef(current_user_content_matrix_cf), 
        index=current_user_content_matrix_cf.index, 
        columns=current_user_content_matrix_cf.index
    )
    print("사용자 유사도 행렬 재계산 완료 (시나리오 1).")
    
    print("\n페르소나 기반 협업 필터링 추천 실행 중 (시나리오 1)...")
    recommended_contents_s1 = recommend_contents_cf(
        target_user_id=temp_user_id_s1,
        user_content_matrix=current_user_content_matrix_cf,
        user_similarity_df=current_user_similarity_df_cf,   
        contents_df_cf=contents_df_cf_for_merge,          
        all_user_personas_df=current_all_user_personas_df,   
        default_personas=default_personas,
        content_genre_keywords_mapping=content_genre_keywords_mapping,
        num_recommendations=10,
        excluded_content_ids=excluded_content_ids_s1 
    )
    
    print(f"\n✨ 사용자 ID {temp_user_id_s1} ('{main_status_text_s1} {main_persona_s1}')를 위한 협업 필터링 추천 작품 (좋아하는 작품 제외):")
    if recommended_contents_s1:
        for i, rec in enumerate(recommended_contents_s1):
            print(f"{i+1}. 제목: {rec['title']} | 장르: {rec['genres']} | 예측 점수: {rec['predicted_rating']:.2f} | 페르소나 장르 매치: {'✅' if rec['persona_genre_match'] else '❌'}")
    else:
        print("추천할 작품을 찾을 수 없습니다. (제외된 작품이 너무 많거나 유사 사용자가 부족할 수 있습니다.)")

else:
    print("콘텐츠 데이터(contents_df_persona)가 없어 시나리오 1을 수행할 수 없습니다.")

새로운 사용자가 질문지에 답변하고 좋아하는 작품을 선택 중...

생성된 통합 사용자 피드백 (시나리오 1): {'스타워즈 에피소드 4: 새로운 희망': 5.0, '메트로폴리스': 5.0, '니모를 찾아서': 5.0}

✅ 시나리오 1 결과:
메인 페르소나: ' 액션_헌터' (점수: 1.80)
서브 페르소나: '이세계_유랑자' (점수: 1.00)
전체 페르소나 점수: {'액션_헌터': '1.80', '무비_셜록': '0.04', '시네파_울보': '0.04', '온기_수집가': '0.12', '이세계_유랑자': '1.00', '무서워도본다맨': '0.00', '레트로_캡틴': '0.00'}

새로운 사용자 포함하여 사용자 유사도 행렬 재계산 중 (시나리오 1)...
사용자 유사도 행렬 재계산 완료 (시나리오 1).

페르소나 기반 협업 필터링 추천 실행 중 (시나리오 1)...


/var/folders/90/psj3mqq143gb_j1tldfm1w7c0000gn/T/ipykernel_2607/488756933.py:8: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_content_ids = pd.concat([user_reactions['contentId'], user_reviews['contentId']]).unique()



✨ 사용자 ID 9999999 (' 액션_헌터')를 위한 협업 필터링 추천 작품 (좋아하는 작품 제외):
1. 제목: 우리 생애 최고의 해 | 장르: 드라마, 전쟁, 로맨스 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
2. 제목: 솔라리스 | 장르: 드라마, 미스터리, SF | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
3. 제목: 영광의 길 | 장르: 드라마, 전쟁, 역사 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
4. 제목: 소프라노스 | 장르: 드라마, 범죄, SF | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
5. 제목: 파프리카 | 장르: 애니메이션, SF, 스릴러 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
6. 제목: 엘리트 스쿼드 | 장르: 드라마, 액션, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
7. 제목: 내 이름은 튜니티 | 장르: 액션, 코미디, 서부 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
8. 제목: 뮬란 | 장르: 모험, 가족, 애니메이션 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
9. 제목: 무간도 | 장르: 미스터리, 범죄, 스릴러, 액션, 드라마 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
10. 제목: 반딧불이의 묘 | 장르: 드라마, 전쟁, 애니메이션 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅


In [20]:
# 시나리오 2: 새로운 사용자 페르소나 정의 및 추천 (질문지 + 별점 기반)

user_answers_s2 = {
    "Q1": "B", # 깊은 감동과 여운
    "Q2": "B", # 인간 관계와 감정선 (시네파_울보)
    "Q3": "C", # 현실적인 이야기 (시네파_울보, 드라마)
    "Q4": "A", # 사랑과 우정 (시네파_울보, 로맨스)
    "Q5": "A"  # 평범한 사람들의 삶 (시네파_울보, 드라마)
}

user_preferred_titles_s2_likes = [
    "라라랜드",
    "노트북",
    "어바웃 타임"
]

print("새로운 사용자가 질문지에 답변하고 별점을 부여 중...")

if not contents_df_persona.empty:
    temp_user_id_s2 = 9999998 

    virtual_content_reactions_df_s2 = pd.DataFrame(columns=['userId', 'contentId', 'reaction']) 

    virtual_content_reactions_df_s2_data = []
    excluded_content_ids_s2 = []

    for title in user_preferred_titles_s2_likes:
        content_info = contents_df_persona[contents_df_persona['title_cleaned'] == clean_title(title)]
        if not content_info.empty:
            content_id = content_info['contentId'].iloc[0]
            virtual_content_reactions_df_s2_data.append({
                'userId': temp_user_id_s2,
                'contentId': content_id,
                'reaction': '좋아요' 
            })
            excluded_content_ids_s2.append(content_id) 

    virtual_content_reactions_df_s2 = pd.DataFrame(virtual_content_reactions_df_s2_data)

    virtual_reviews_df_s2 = pd.DataFrame(columns=['userId', 'contentId', 'score'])

    unified_user_feedback_s2 = generate_unified_user_feedback(
        temp_user_id_s2,
        virtual_content_reactions_df_s2, 
        virtual_reviews_df_s2,          
        contents_df_persona
    )

    print(f"\n생성된 통합 사용자 피드백 (시나리오 2): {unified_user_feedback_s2}")

    main_persona_s2, main_score_s2, all_scores_s2, sub_persona_s2, sub_score_s2, main_status_text_s2 = get_hybrid_persona(
        user_answers_s2,
        unified_user_feedback_s2,
        contents_df_persona,
        default_personas,
        content_genre_keywords_mapping,
        qa_weight_factor=0.5,
        min_positive_ratings_for_growth=MIN_POSITIVE_RATINGS_FOR_GROWTH,
        min_score_for_growth=MIN_SCORE_FOR_GROWTH,
        positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD
    )

    print(f"\n✅ 시나리오 2 결과:")
    print(f"메인 페르소나: '{main_status_text_s2} {main_persona_s2}' (점수: {main_score_s2:.2f})")
    if sub_persona_s2:
        print(f"서브 페르소나: '{sub_persona_s2}' (점수: {sub_score_s2:.2f})")
    print("전체 페르소나 점수:", {k: f"{v:.2f}" for k, v in all_scores_s2.items()})

    
    new_user_persona_data_s2 = {
        'userId': temp_user_id_s2,
        'main_persona': main_persona_s2,
        'main_persona_score': main_score_s2,
        'sub_persona': sub_persona_s2,
        'sub_persona_score': sub_score_s2,
        'persona_status': main_status_text_s2
    }

    current_all_user_personas_df = pd.concat([current_all_user_personas_df, pd.DataFrame([new_user_persona_data_s2])], ignore_index=True)


    current_user_content_matrix_cf = current_user_content_matrix_cf.copy() 

    if temp_user_id_s2 not in current_user_content_matrix_cf.index:
        current_user_content_matrix_cf.loc[temp_user_id_s2] = 0.0

    for title_cleaned, score in unified_user_feedback_s2.items():
        content_id_row = contents_df_persona[contents_df_persona['title_cleaned'] == title_cleaned]
        if not content_id_row.empty:
            content_id = content_id_row['contentId'].iloc[0]
            if content_id in current_user_content_matrix_cf.columns:
                current_user_content_matrix_cf.loc[temp_user_id_s2, content_id] = score
            else:
                current_user_content_matrix_cf[content_id] = 0.0
                current_user_content_matrix_cf.loc[temp_user_id_s2, content_id] = score

    print("\n새로운 사용자 포함하여 사용자 유사도 행렬 재계산 중 (시나리오 2)...")
    current_user_similarity_df_cf = pd.DataFrame(
        np.corrcoef(current_user_content_matrix_cf),
        index=current_user_content_matrix_cf.index,
        columns=current_user_content_matrix_cf.index
    )
    print("사용자 유사도 행렬 재계산 완료 (시나리오 2).")

    print("\n페르소나 기반 협업 필터링 추천 실행 중 (시나리오 2)...")
    recommended_contents_s2 = recommend_contents_cf(
        target_user_id=temp_user_id_s2,
        user_content_matrix=current_user_content_matrix_cf,
        user_similarity_df=current_user_similarity_df_cf,
        contents_df_cf=contents_df_cf_for_merge,
        all_user_personas_df=current_all_user_personas_df,
        default_personas=default_personas,
        content_genre_keywords_mapping=content_genre_keywords_mapping,
        num_recommendations=10,
        excluded_content_ids=excluded_content_ids_s2
    )

    print(f"\n✨ 사용자 ID {temp_user_id_s2} ('{main_status_text_s2} {main_persona_s2}')를 위한 협업 필터링 추천 작품 (이미 본 작품 제외):")
    if recommended_contents_s2:
        for i, rec in enumerate(recommended_contents_s2):
            print(f"{i+1}. 제목: {rec['title']} | 장르: {rec['genres']} | 예측 점수: {rec['predicted_rating']:.2f} | 페르소나 장르 매치: {'✅' if rec['persona_genre_match'] else '❌'}")
    else:
        print("추천할 작품을 찾을 수 없습니다. (제외된 작품이 너무 많거나 유사 사용자가 부족할 수 있습니다.)")

else:
    print("콘텐츠 데이터(contents_df_persona)가 없어 시나리오 2를 수행할 수 없습니다.")

새로운 사용자가 질문지에 답변하고 별점을 부여 중...

생성된 통합 사용자 피드백 (시나리오 2): {'라라랜드': 5.0, '노트북': 5.0, '어바웃 타임': 5.0}

✅ 시나리오 2 결과:
메인 페르소나: ' 온기_수집가' (점수: 0.93)
서브 페르소나: '레트로_캡틴' (점수: 0.73)
전체 페르소나 점수: {'액션_헌터': '0.00', '무비_셜록': '0.59', '시네파_울보': '0.37', '온기_수집가': '0.93', '이세계_유랑자': '0.03', '무서워도본다맨': '0.35', '레트로_캡틴': '0.73'}

새로운 사용자 포함하여 사용자 유사도 행렬 재계산 중 (시나리오 2)...
사용자 유사도 행렬 재계산 완료 (시나리오 2).

페르소나 기반 협업 필터링 추천 실행 중 (시나리오 2)...

✨ 사용자 ID 9999998 (' 온기_수집가')를 위한 협업 필터링 추천 작품 (이미 본 작품 제외):
1. 제목: 죽은 시인의 사회 | 장르: 드라마 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
2. 제목: 배트맨 비긴즈 | 장르: 드라마, 액션, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
3. 제목: 아마데우스 | 장르: 드라마, 음악, 역사 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
4. 제목: 인생은 아름다워 | 장르: 드라마, 코미디 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
5. 제목: What's New, Scooby-Doo? | 장르: 미스터리, 전쟁, 액션, 역사, 액션&모험, 애니메이션 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
6. 제목: Chappelle's Show | 장르: 코미디 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
7. 제목: 나 홀로 집에 2: 뉴욕을 헤매다 | 장르: 모험, 코미디, 가족 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
8. 제목: 스파이더맨 | 장르: 애니메이션, 액션&모험 | 예측 점수: 5.0

/var/folders/90/psj3mqq143gb_j1tldfm1w7c0000gn/T/ipykernel_2607/488756933.py:8: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_content_ids = pd.concat([user_reactions['contentId'], user_reviews['contentId']]).unique()


In [21]:
user_answers_s3 = {
    "Q1": "C", # 유쾌하고 따뜻한 웃음
    "Q2": "C", # 마음이 따뜻해지는 감동 (온기_수집가)
    "Q3": "A", # 상상력을 자극하는 판타지 세계 (이세계_유랑자)
    "Q4": "B", # 동물이나 자연을 배경 (온기_수집가)
    "Q5": "C"  # 밝고 긍정적인 메시지 (온기_수집가)
}

user_initial_preference_actions_s3 = [
    {"title": "쏘우", "reaction": "좋아요", "score": None},          # 공포
    {"title": "언터처블", "reaction": "좋아요", "score": None}, # 범죄, 스릴러
    {"title": "Only Fools and Horses", "reaction": "좋아요", "score": None}     # 코미디
]

print("새로운 사용자가 질문지에 답변하고 선호하는 작품을 선택 중 ('좋아요' 반응 시뮬레이션)...")

if not contents_df_persona.empty:
    temp_user_id_s3 = 9999997

    virtual_content_reactions_s3_data = []
    virtual_reviews_s3_data = [] # 시나리오 3에서는 이 리스트가 비어있을 것입니다.
    excluded_content_ids_s3 = [] 

    for action in user_initial_preference_actions_s3:
        title = action['title']
        content_info = contents_df_persona[contents_df_persona['title_cleaned'] == clean_title(title)]
        
        if not content_info.empty:
            content_id = content_info['contentId'].iloc[0]
            
            if action['reaction'] == '좋아요':
                virtual_content_reactions_s3_data.append({
                    'userId': temp_user_id_s3,
                    'contentId': content_id,
                    'reaction': '좋아요' 
                })
            # 시나리오 3에서는 '평점' 액션이 없으므로, virtual_reviews_s3_data는 비어있게 됩니다.
            
            if content_id not in excluded_content_ids_s3:
                excluded_content_ids_s3.append(content_id)
        else:
            print(f"경고: '{title}'에 해당하는 콘텐츠를 찾을 수 없습니다. 이 작품에 대한 피드백은 무시됩니다.")

    virtual_content_reactions_df_s3 = pd.DataFrame(virtual_content_reactions_s3_data, columns=['userId', 'contentId', 'reaction'])
    # virtual_reviews_df_s3는 의도적으로 비어있는 DataFrame으로 생성됩니다.
    virtual_reviews_df_s3 = pd.DataFrame(virtual_reviews_s3_data, columns=['userId', 'contentId', 'score'])
    
    unified_user_feedback_s3 = generate_unified_user_feedback(
        temp_user_id_s3,
        virtual_content_reactions_df_s3,
        virtual_reviews_df_s3,
        contents_df_persona
    )

    print(f"\n생성된 통합 사용자 피드백 (시나리오 3): {unified_user_feedback_s3}")

    main_persona_s3, main_score_s3, all_scores_s3, sub_persona_s3, sub_score_s3, main_status_text_s3 = get_hybrid_persona(
        user_answers_s3,
        unified_user_feedback_s3,
        contents_df_persona,
        default_personas,
        content_genre_keywords_mapping,
        qa_weight_factor=0.5,
        min_positive_ratings_for_growth=MIN_POSITIVE_RATINGS_FOR_GROWTH,
        min_score_for_growth=MIN_SCORE_FOR_GROWTH,
        positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD
    )

    print(f"\n✅ 시나리오 3 결과:")
    print(f"메인 페르소나: '{main_status_text_s3} {main_persona_s3}' (점수: {main_score_s3:.2f})")
    if sub_persona_s3:
        print(f"서브 페르소나: '{sub_persona_s3}' (점수: {sub_score_s3:.2f})")
    print("전체 페르소나 점수:", {k: f"{v:.2f}" for k, v in all_scores_s3.items()})

    new_user_persona_data_s3 = {
        'userId': temp_user_id_s3,
        'main_persona': main_persona_s3,
        'main_persona_score': main_score_s3,
        'sub_persona': sub_persona_s3,
        'sub_persona_score': sub_score_s3,
        'persona_status': main_status_text_s3
    }
    
    if 'current_all_user_personas_df' in locals():
         current_all_user_personas_df = pd.concat([current_all_user_personas_df, pd.DataFrame([new_user_persona_data_s3])], ignore_index=True)
    elif 'all_user_personas_df' in locals():
         current_all_user_personas_df = pd.concat([all_user_personas_df, pd.DataFrame([new_user_persona_data_s3])], ignore_index=True)
    else:
        print("[경고] 'current_all_user_personas_df' 또는 'all_user_personas_df'를 찾을 수 없습니다. 새로운 사용자 페르소나 정보만으로 진행합니다.")
        current_all_user_personas_df = pd.DataFrame([new_user_persona_data_s3])

    if 'current_user_content_matrix_cf' not in locals() and 'user_content_matrix_cf' in locals():
        current_user_content_matrix_cf = user_content_matrix_cf.copy()
    elif 'current_user_content_matrix_cf' not in locals():
        print("[경고] 'current_user_content_matrix_cf' 또는 'user_content_matrix_cf'를 찾을 수 없습니다. 새로운 사용자 데이터만으로 행렬을 생성합니다.")
        current_user_content_matrix_cf = pd.DataFrame(columns=contents_df_persona['contentId'].unique(), index=[temp_user_id_s3]).fillna(0)


    if temp_user_id_s3 not in current_user_content_matrix_cf.index:
        current_user_content_matrix_cf.loc[temp_user_id_s3] = 0.0

    for title_cleaned, score in unified_user_feedback_s3.items():
        content_id_row = contents_df_persona[contents_df_persona['title_cleaned'] == clean_title(title_cleaned)]
        if not content_id_row.empty:
            content_id = content_id_row['contentId'].iloc[0]
            if content_id in current_user_content_matrix_cf.columns:
                current_user_content_matrix_cf.loc[temp_user_id_s3, content_id] = score
            else:
                current_user_content_matrix_cf[content_id] = 0.0 
                current_user_content_matrix_cf.loc[temp_user_id_s3, content_id] = score

    print("\n새로운 사용자 포함하여 사용자 유사도 행렬 재계산 중 (시나리오 3)...")
    if current_user_content_matrix_cf.shape[0] > 1:
        current_user_similarity_df_cf = pd.DataFrame(
            np.corrcoef(current_user_content_matrix_cf),
            index=current_user_content_matrix_cf.index,
            columns=current_user_content_matrix_cf.index
        )
        current_user_similarity_df_cf.fillna(0, inplace=True) 
        print("사용자 유사도 행렬 재계산 완료 (시나리오 3).")
    else:
        print("[경고] 사용자가 한 명뿐이므로 유사도 행렬을 재계산할 수 없습니다. 빈 유사도 행렬 사용.")
        current_user_similarity_df_cf = pd.DataFrame(index=[temp_user_id_s3], columns=[temp_user_id_s3], data=1.0) 

    print("\n페르소나 기반 협업 필터링 추천 실행 중 (시나리오 3)...")
    recommended_contents_s3 = recommend_contents_cf(
        target_user_id=temp_user_id_s3,
        user_content_matrix=current_user_content_matrix_cf,
        user_similarity_df=current_user_similarity_df_cf,
        contents_df_cf=contents_df_cf_for_merge,
        all_user_personas_df=current_all_user_personas_df,
        default_personas=default_personas,
        content_genre_keywords_mapping=content_genre_keywords_mapping,
        num_recommendations=10,
        excluded_content_ids=excluded_content_ids_s3
    )

    print(f"\n✨ 사용자 ID {temp_user_id_s3} ('{main_status_text_s3} {main_persona_s3}')를 위한 협업 필터링 추천 작품 (이미 본 작품 제외):")
    if recommended_contents_s3:
        for i, rec in enumerate(recommended_contents_s3):
            print(f"{i+1}. 제목: {rec['title']} | 장르: {rec['genres']} | 예측 점수: {rec['predicted_rating']:.2f} | 페르소나 장르 매치: {'✅' if rec['persona_genre_match'] else '❌'}")
    else:
        print("추천할 작품을 찾을 수 없습니다. (제외된 작품이 너무 많거나 유사 사용자가 부족할 수 있습니다.)")

else:
    print("콘텐츠 데이터(contents_df_persona)가 없어 시나리오 3을 수행할 수 없습니다.")

새로운 사용자가 질문지에 답변하고 선호하는 작품을 선택 중 ('좋아요' 반응 시뮬레이션)...

생성된 통합 사용자 피드백 (시나리오 3): {'쏘우': 5.0, '언터처블': 5.0, 'Only Fools and Horses': 5.0}

✅ 시나리오 3 결과:
메인 페르소나: ' 무비_셜록' (점수: 1.33)
서브 페르소나: '시네파_울보' (점수: 0.63)
전체 페르소나 점수: {'액션_헌터': '0.10', '무비_셜록': '1.33', '시네파_울보': '0.63', '온기_수집가': '0.23', '이세계_유랑자': '0.00', '무서워도본다맨': '0.47', '레트로_캡틴': '0.23'}

새로운 사용자 포함하여 사용자 유사도 행렬 재계산 중 (시나리오 3)...
사용자 유사도 행렬 재계산 완료 (시나리오 3).

페르소나 기반 협업 필터링 추천 실행 중 (시나리오 3)...

✨ 사용자 ID 9999997 (' 무비_셜록')를 위한 협업 필터링 추천 작품 (이미 본 작품 제외):
1. 제목: 미녀와 뱀파이어 | 장르: 드라마, SF&판타지, 코미디, 액션&모험 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
2. 제목: 스탠 바이 미 | 장르: 드라마, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
3. 제목: 그린 마일 | 장르: 판타지, 드라마, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
4. 제목: 미저리 | 장르: 드라마, 스릴러 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
5. 제목: 조찬 클럽 | 장르: 드라마, 코미디 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
6. 제목: 스크림 | 장르: 범죄, 미스터리, 공포 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
7. 제목: 이터널 선샤인 | 장르: 로맨스, 드라마, SF | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
8. 제목: 글래디에이터 | 장르: 드라마, 액션, 모험 | 예측 점수: 5.00 | 페르소나 

/var/folders/90/psj3mqq143gb_j1tldfm1w7c0000gn/T/ipykernel_2607/488756933.py:8: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_content_ids = pd.concat([user_reactions['contentId'], user_reviews['contentId']]).unique()


In [23]:

def handle_user_feedback_and_update_recommendations(user_id, content_id, reaction_type, score=None, user_answers=None):
    global current_virtual_reactions_df, current_virtual_reviews_df, \
           current_all_user_personas_df, current_user_content_matrix_cf, \
           current_user_similarity_df_cf, contents_df_persona, contents_df_cf_for_merge, \
           default_personas, content_genre_keywords_mapping, \
           LIKE_WEIGHT, DISLIKE_WEIGHT, STAR_RATING_WEIGHTS, \
           MIN_POSITIVE_RATINGS_FOR_GROWTH, MIN_SCORE_FOR_GROWTH, POSITIVE_FEEDBACK_THRESHOLD, \
           THRESHOLD_FOR_BABY_PERSONA

    print(f"\n사용자 ID {user_id}의 새로운 피드백 처리 중: 콘텐츠 {content_id}, 유형: {reaction_type}, 점수: {score}")

    # 1. 새로운 피드백 데이터 추가
    if reaction_type in ['좋아요', '싫어요']:
        existing_entry_idx = current_virtual_reactions_df[
            (current_virtual_reactions_df['userId'] == user_id) &
            (current_virtual_reactions_df['contentId'] == content_id)
        ].index
        if not existing_entry_idx.empty:
            current_virtual_reactions_df.loc[existing_entry_idx, 'reaction'] = reaction_type
        else:
            new_feedback = pd.DataFrame([{
                'userId': user_id,
                'contentId': content_id,
                'reaction': reaction_type
            }])
            current_virtual_reactions_df = pd.concat([current_virtual_reactions_df, new_feedback], ignore_index=True)

    elif reaction_type == '평점' and score is not None:
        existing_entry_idx = current_virtual_reviews_df[
            (current_virtual_reviews_df['userId'] == user_id) &
            (current_virtual_reviews_df['contentId'] == content_id)
        ].index
        if not existing_entry_idx.empty:
            current_virtual_reviews_df.loc[existing_entry_idx, 'score'] = score
        else:
            new_feedback = pd.DataFrame([{
                'userId': user_id,
                'contentId': content_id,
                'score': score
            }])
            current_virtual_reviews_df = pd.concat([current_virtual_reviews_df, new_feedback], ignore_index=True)

    # 2. 통합 사용자 피드백 재계산
    unified_user_feedback = generate_unified_user_feedback(
        user_id,
        current_virtual_reactions_df[current_virtual_reactions_df['userId'] == user_id],
        current_virtual_reviews_df[current_virtual_reviews_df['userId'] == user_id],
        contents_df_persona
    )

    # 3. 사용자 페르소나 업데이트 (또는 초기화)
    if user_answers is None:
        current_user_answers = globals().get('user_answers_s3', {}) 
    else:
        current_user_answers = user_answers

    main_persona, main_score, all_scores, sub_persona, sub_score, main_status_text = get_hybrid_persona(
        current_user_answers,
        unified_user_feedback,
        contents_df_persona,
        default_personas,
        content_genre_keywords_mapping,
        qa_weight_factor=0.5,
        min_positive_ratings_for_growth=MIN_POSITIVE_RATINGS_FOR_GROWTH,
        min_score_for_growth=MIN_SCORE_FOR_GROWTH,
        positive_feedback_threshold=POSITIVE_FEEDBACK_THRESHOLD
    )

    user_persona_data = {
        'userId': user_id,
        'main_persona': main_persona,
        'main_persona_score': main_score,
        'sub_persona': sub_persona,
        'sub_persona_score': sub_score,
        'persona_status': main_status_text
    }
    user_idx = current_all_user_personas_df[current_all_user_personas_df['userId'] == user_id].index
    if not user_idx.empty:
        for key, value in user_persona_data.items():
            current_all_user_personas_df.loc[user_idx[0], key] = value
    else: 
        current_all_user_personas_df = pd.concat([current_all_user_personas_df, pd.DataFrame([user_persona_data])], ignore_index=True)

    print(f"업데이트된 페르소나: '{main_status_text} {main_persona}' (점수: {main_score:.2f})")

    # 4. 사용자-콘텐츠 행렬 업데이트
    if user_id not in current_user_content_matrix_cf.index:
        current_user_content_matrix_cf.loc[user_id] = 0.0 

    for title_cleaned_feedback, score_feedback in unified_user_feedback.items():
        content_info_row = contents_df_persona[contents_df_persona['title_cleaned'] == title_cleaned_feedback]
        if not content_info_row.empty:
            c_id = content_info_row['contentId'].iloc[0]
            if c_id not in current_user_content_matrix_cf.columns:
                current_user_content_matrix_cf[c_id] = 0.0 
            current_user_content_matrix_cf.loc[user_id, c_id] = score_feedback

    # 5. 사용자 유사도 행렬 재계산
    if current_user_content_matrix_cf.shape[0] > 1 and current_user_content_matrix_cf.shape[1] > 1:
        current_user_similarity_df_cf = pd.DataFrame(
            np.corrcoef(current_user_content_matrix_cf),
            index=current_user_content_matrix_cf.index,
            columns=current_user_content_matrix_cf.index
        )
        current_user_similarity_df_cf.fillna(0, inplace=True)
        print("사용자 유사도 행렬 재계산 완료.")
    elif current_user_content_matrix_cf.shape[0] == 1:
        current_user_similarity_df_cf = pd.DataFrame(index=[user_id], columns=[user_id], data=1.0)
    else:
        current_user_similarity_df_cf = pd.DataFrame() 
        print("[경고] 사용자 또는 콘텐츠 데이터가 부족하여 유사도 행렬을 유의미하게 계산할 수 없습니다. 빈 유사도 행렬 사용.")


    # 6. 추천 목록 생성 및 반환/캐시
    excluded_content_ids_for_reco = list(current_virtual_reactions_df[current_virtual_reactions_df['userId'] == user_id]['contentId'].unique()) + \
                                   list(current_virtual_reviews_df[current_virtual_reviews_df['userId'] == user_id]['contentId'].unique())
    excluded_content_ids_for_reco = list(set(excluded_content_ids_for_reco)) # 중복 제거

    recommended_contents = recommend_contents_cf(
            target_user_id=user_id,
            user_content_matrix=current_user_content_matrix_cf,
            user_similarity_df=current_user_similarity_df_cf,
            contents_df_cf=contents_df_cf_for_merge,
            all_user_personas_df=current_all_user_personas_df,
            default_personas=default_personas,
            content_genre_keywords_mapping=content_genre_keywords_mapping,
            num_recommendations=10,
            excluded_content_ids=excluded_content_ids_for_reco
        )

    # 협업 필터링 추천이 비어있으면 콘텐츠 기반 추천으로 폴백
    if not recommended_contents:
        print("[정보] 협업 필터링 추천이 충분하지 않아 콘텐츠 기반 추천을 시도합니다.")
        
        # 사용자가 가장 최근에 긍정적으로 반응한 콘텐츠를 찾습니다.
        user_feedback_for_content_based = {}
        for title_cleaned, score in unified_user_feedback.items():
            if score > POSITIVE_FEEDBACK_THRESHOLD:
                user_feedback_for_content_based[title_cleaned] = score
        
        # 가장 점수가 높은 (최근 좋아요/높은 평점) 콘텐츠를 선택합니다.
        if user_feedback_for_content_based:
            most_liked_title = max(user_feedback_for_content_based, key=user_feedback_for_content_based.get)
            content_info = contents_df_persona[contents_df_persona['title_cleaned'] == most_liked_title]
            if not content_info.empty:
                target_content_id_for_cb = content_info['contentId'].iloc[0]
                recommended_contents = recommend_contents_content_based(
                    target_content_ids=[target_content_id_for_cb],
                    content_similarity_df=content_similarity_df, 
                    contents_df=contents_df_persona, 
                    num_recommendations=10
                )
                
                for rec in recommended_contents:
                    rec['predicted_rating'] = rec['similarity'] 
                    rec['persona_genre_match'] = True 
                
                print(f"[정보] '{most_liked_title}' 기반 콘텐츠 유사도 추천: {len(recommended_contents)}개")
            else:
                print("[경고] 콘텐츠 기반 추천을 위한 타겟 콘텐츠 정보를 찾을 수 없습니다.")
        else:
            print("[경고] 콘텐츠 기반 추천을 위한 긍정적인 피드백이 없습니다.")


    print(f"\n✨ 사용자 ID {user_id}를 위한 새로운 추천 작품 (이미 본 작품 제외):")
    if recommended_contents:
        for i, rec in enumerate(recommended_contents):
            sim_score = rec.get('similarity', rec.get('predicted_rating')) 
            print(f"{i+1}. 제목: {rec['title']} | 장르: {rec['genres']} | 예측 점수: {rec['predicted_rating']:.2f} | 페르소나 장르 매치: {'✅' if rec['persona_genre_match'] else '❌'}")
    else:
        print("추천할 작품을 찾을 수 없습니다.")

    return recommended_contents

In [24]:
current_virtual_reactions_df = pd.DataFrame(columns=['userId', 'contentId', 'reaction'])
current_virtual_reviews_df = pd.DataFrame(columns=['userId', 'contentId', 'score'])
current_all_user_personas_df = all_user_personas_df.copy() 
current_user_content_matrix_cf = user_content_matrix_cf.copy() 
current_user_similarity_df_cf = user_similarity_df_cf.copy() 

user_answers_s3 = { # 시나리오 3에서 정의했던 것과 동일
    "Q1": "C", # 유쾌하고 따뜻한 웃음
    "Q2": "C", # 마음이 따뜻해지는 감동 (온기_수집가)
    "Q3": "A", # 상상력을 자극하는 판타지 세계 (이세계_유랑자)
    "Q4": "B", # 동물이나 자연을 배경 (온기_수집가)
    "Q5": "C"  # 밝고 긍정적인 메시지 (온기_수집가)
}
temp_user_id_s3 = 9999997 # 시나리오 3에서 사용된 ID와 동일

print("--- 시나리오 3 초기 피드백 처리 시작 (handle_user_feedback_and_update_recommendations 함수 사용) ---")
initial_actions_s3 = [ # 시나리오 3 초기 좋아요 피드백
    {"title": "쏘우", "reaction": "좋아요", "score": None},
    {"title": "언터처블", "reaction": "좋아요", "score": None},
    {"title": "Only Fools and Horses", "reaction": "좋아요", "score": None}
]

for action in initial_actions_s3:
    content_info = contents_df_persona[contents_df_persona['title_cleaned'] == clean_title(action['title'])]
    if not content_info.empty:
        content_id_val = content_info['contentId'].iloc[0]
        handle_user_feedback_and_update_recommendations(
            user_id=temp_user_id_s3,
            content_id=content_id_val,
            reaction_type=action['reaction'],
            score=action['score'],
            user_answers=user_answers_s3 # 최초 등록 시에만 질문지 답변 전달
        )
    else:
        print(f"초기 피드백 경고: '{action['title']}'에 해당하는 콘텐츠를 찾을 수 없습니다.")
print("--- 시나리오 3 초기 피드백 처리 완료 ---")

print("\n--- 추가 사용자 피드백 처리 시작 (페르소나 변경 시뮬레이션) ---")
additional_actions_s4_replacement = [ 
    {"title": "인사이드 아웃", "reaction": "별점", "score": 5.0},
    {"title": "토이 스토리", "reaction": "별점", "score": 5.0},
    {"title": "업", "reaction": "별점", "score": 5.0},
    {"title": "주토피아", "reaction": "별점", "score": 5.0},
    {"title": "겨울왕국", "reaction": "별점", "score": 5.0},
    {"title": "슈렉", "reaction": "별점", "score": 5.0},
    {"title": "몬스터 주식회사", "reaction": "별점", "score": 5.0},
    {"title": "빅 히어로", "reaction": "별점", "score": 5.0},
    {"title": "모아나", "reaction": "별점", "score": 5.0},
    {"title": "코코", "reaction": "별점", "score": 5.0},
    {"title": "해리 포터와 마법사의 돌", "reaction": "별점", "score": 5.0},
    {"title": "반지의 제왕: 반지 원정대", "reaction": "별점", "score": 5.0},
    {"title": "이상한 나라의 앨리스", "reaction": "별점", "score": 5.0},
    {"title": "나니아 연대기: 사자, 마녀 그리고 옷장", "reaction": "별점", "score": 5.0},
    {"title": "닥터 후", "reaction": "좋아요", "score": None},
    {"title": "아바타 아앙의 전설", "reaction": "별점", "score": 5.0},
    {"title": "SMTV Live", "reaction": "좋아요", "score": None},
]

for action in additional_actions_s4_replacement:
    content_info = contents_df_persona[contents_df_persona['title_cleaned'] == clean_title(action['title'])]
    if not content_info.empty:
        content_id_val = content_info['contentId'].iloc[0]
        handle_user_feedback_and_update_recommendations(
            user_id=temp_user_id_s3,
            content_id=content_id_val,
            reaction_type=action['reaction'],
            score=action['score']
            # user_answers는 이미 등록된 사용자이므로 전달할 필요 없음
        )
    else:
        print(f"추가 피드백 경고: '{action['title']}'에 해당하는 콘텐츠를 찾을 수 없습니다.")
print("--- 추가 사용자 피드백 처리 완료 ---")

--- 시나리오 3 초기 피드백 처리 시작 (handle_user_feedback_and_update_recommendations 함수 사용) ---

사용자 ID 9999997의 새로운 피드백 처리 중: 콘텐츠 176, 유형: 좋아요, 점수: None
업데이트된 페르소나: ' 무비_셜록' (점수: 1.37)
사용자 유사도 행렬 재계산 완료.

✨ 사용자 ID 9999997를 위한 새로운 추천 작품 (이미 본 작품 제외):
1. 제목: 뻐꾸기 둥지 위로 날아간 새 | 장르: 드라마 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
2. 제목: 좋은 친구들 | 장르: 드라마, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
3. 제목: 몽상가들 | 장르: 드라마, 로맨스 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
4. 제목: 길버트 그레이프 | 장르: 드라마, 로맨스 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
5. 제목: 브롱스 이야기 | 장르: 드라마, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
6. 제목: 문라이즈 킹덤 | 장르: 드라마, 코미디, 로맨스 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
7. 제목: 베이비 드라이버 | 장르: 액션, 범죄 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
8. 제목: 글래디에이터 | 장르: 드라마, 액션, 모험 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
9. 제목: What's New, Scooby-Doo? | 장르: 미스터리, 전쟁, 액션, 역사, 액션&모험, 애니메이션 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅
10. 제목: 라이언 일병 구하기 | 장르: 드라마, 전쟁, 역사 | 예측 점수: 5.00 | 페르소나 장르 매치: ✅

사용자 ID 9999997의 새로운 피드백 처리 중: 콘텐츠 117, 유형: 좋아요, 점수: None
업데이트된 페르소나: ' 무비_셜록' (점수: 1.34)
사용자 유사도 행렬 재계산 완료.

✨ 